In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns; sns.set()
import csv

from PIL import Image, ImageDraw
import os
import shutil

from mpl_toolkits.basemap import Basemap

#there is currently a bug in panda's scatter_matrix that produces a warning that has no affect on what I'm doin :)
import warnings
warnings.filterwarnings("ignore")

In [2]:
#load in the data
df = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False, delim_whitespace=True)
df.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)
df_center = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_Reduced_Trackfile.txt',header=None,names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"],low_memory=False,sep='\t')
df_center.dropna(axis=0,how='any',subset=['Lat','Long'],inplace=True)

df_time = pd.DataFrame()
df_time['cluster'] = 0

df_inner = df.copy()

In [3]:
display(df.head())
display(df_center.head())
display(df_time.head())

,Year,Month,Day,Hour,Min,Sec,Lat,Long,Dist_East_West,Dist_North_South
0,2017,8,27,17,56,9.7207,12.7452,-13.4960,271.5710,138.4600
1,2017,8,27,17,56,19.4011,12.0782,-16.2206,-23.9866,64.2929
2,2017,8,27,17,56,36.8910,9.6958,-8.5119,820.7290,-200.6180
3,2017,8,27,17,56,42.7955,10.8790,-10.6200,587.4700,-69.0520
4,2017,8,27,17,56,46.1723,9.9119,-12.6214,370.0740,-176.5890


,Year,Month,Day,Hour,Lat,Long,Min_Pressure,Max_Winds,Unused
0,2017,8,27,18,11.5,-16.0,0,25,0
1,2017,8,28,0,11.5,-17.8,0,25,0
2,2017,8,28,6,11.6,-19.3,1009,25,0
3,2017,8,28,12,11.9,-20.5,1009,25,0
4,2017,8,28,18,12.3,-21.0,1008,25,0


,cluster


In [4]:
#make a dataframe that contains strikes in time intervals
minutes_between = 30
cluster = 0

for x in range(len(df['Month'].unique())):
    month = df['Month'].unique()[x]
    for y in range(len(df[df['Month'] == df['Month'].unique()[x]]['Day'].unique())):
        day = df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]
        for z in range(len(df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique())):
            hour = df[df['Day'] == df[df['Month'] == df['Month'].unique()[x]]['Day'].unique()[y]]['Hour'].unique()[z]
            for w in range(0,60,minutes_between):
                minute = w
                aeon_sucks = df[(df['Month'] == month) & (df['Day'] == day) & (df['Hour'] == hour) & (df['Min'] >= minute) & (df['Min'] <= minute + minutes_between)]
                aeon_sucks['cluster'] = cluster
                df_time = df_time.append(aeon_sucks, ignore_index=True)
                cluster+=1

display(df_time)
display(df_time.tail())

,Day,Dist_East_West,Dist_North_South,Hour,Lat,Long,Min,Month,Sec,Year,cluster
0,27.0,271.5710,138.46000,17.0,12.7452,-13.4960,56.0,8.0,9.7207,2017.0,1
1,27.0,-23.9866,64.29290,17.0,12.0782,-16.2206,56.0,8.0,19.4011,2017.0,1
2,27.0,820.7290,-200.61800,17.0,9.6958,-8.5119,56.0,8.0,36.8910,2017.0,1
3,27.0,587.4700,-69.05200,17.0,10.8790,-10.6200,56.0,8.0,42.7955,2017.0,1
4,27.0,370.0740,-176.58900,17.0,9.9119,-12.6214,56.0,8.0,46.1723,2017.0,1
5,27.0,-142.3300,-154.37200,17.0,10.1117,-17.3002,56.0,8.0,47.9845,2017.0,1
6,27.0,-139.4640,-154.88300,17.0,10.1071,-17.2740,56.0,8.0,47.9846,2017.0,1
7,27.0,304.8490,121.90300,17.0,12.5963,-13.1908,56.0,8.0,48.2625,2017.0,1
8,27.0,383.8270,-158.67500,17.0,10.0730,-12.4941,57.0,8.0,6.6318,2017.0,1
9,27.0,381.2760,-155.58400,17.0,10.1008,-12.5171,57.0,8.0,6.6318,2017.0,1


,Day,Dist_East_West,Dist_North_South,Hour,Lat,Long,Min,Month,Sec,Year,cluster
230715,11.0,529.132,-248.19800,18.0,28.6679,-78.0761,3.0,9.0,42.9437,2017.0,722
230716,11.0,646.593,-957.66600,18.0,22.2875,-77.2151,3.0,9.0,48.6514,2017.0,722
230717,11.0,484.965,-2.43517,18.0,30.8781,-78.4179,4.0,9.0,0.5648,2017.0,722
230718,11.0,529.056,-244.80700,18.0,28.6984,-78.0753,4.0,9.0,26.9945,2017.0,722
230719,11.0,871.952,-1023.76000,18.0,21.6931,-75.0596,4.0,9.0,55.8509,2017.0,722


In [5]:
df_inner['distance'] = np.sqrt((df_inner['Dist_East_West'] * df_inner['Dist_East_West']) + 
                               (df_inner['Dist_North_South'] * df_inner['Dist_North_South']))
df_inner = df_inner[df_inner['distance'] <= 100]
df_inner

,Year,Month,Day,Hour,Min,Sec,Lat,Long,Dist_East_West,Dist_North_South,distance
1,2017,8,27,17,56,19.4011,12.0782,-16.2206,-23.98660,64.29290,68.621673
36,2017,8,27,17,59,33.2243,12.0928,-16.2278,-24.76810,65.91640,70.416124
49,2017,8,27,18,1,19.4126,12.0947,-16.2925,-31.80260,66.12760,73.377550
76,2017,8,27,18,2,49.9460,12.1423,-16.1575,-17.12140,71.42050,73.444061
104,2017,8,27,18,4,35.0135,12.0755,-16.3007,-32.69650,63.99270,71.861859
105,2017,8,27,18,4,35.0135,12.0660,-16.2787,-30.30540,62.93630,69.852667
166,2017,8,27,18,8,43.8543,12.0908,-16.2972,-26.27200,65.67060,70.730797
167,2017,8,27,18,8,43.8543,12.1007,-16.2628,-22.53090,66.77140,70.470287
177,2017,8,27,18,9,45.2420,12.1170,-16.2763,-23.99720,68.58390,72.660973
221,2017,8,27,18,13,44.1413,12.0955,-16.3173,-28.45690,66.19320,72.050919


In [ ]:
coord_df = df.loc[:,['Lat','Long']]
coord_df.head()

In [ ]:
#elbow graph for visual estimation of ideal cluster count
clusters = range(1,16)

kmeans = [KMeans(n_clusters=i) for i in clusters]

y = df[['Lat']]
x = df[['Long']]

score = [kmeans[i].fit(y).score(y) for i in range(len(kmeans))]

plt.plot(clusters, score)

plt.show()

In [ ]:
cluster_df = coord_df
for n_clusters in range(1,16):
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++')
    kmeans.fit(cluster_df)
    inertia = kmeans.inertia_
    print("Cluster count: ", n_clusters, "\tInertia: ", inertia)

In [ ]:
cluster_df.plot.scatter(x='Lat',y='Long', c='pink', s=50, cmap='spring')
plt.show()

In [ ]:
#kmeans using the set cluster count
kmeans = KMeans(n_clusters=4, init='k-means++')
cluster_df = coord_df
kmeans.fit(cluster_df)
cluster_df['cluster'] = kmeans.fit_predict(cluster_df)
inertia = kmeans.inertia_
labels = kmeans.predict(cluster_df[cluster_df.columns[:2]])
centers = kmeans.cluster_centers_

cluster_df.plot.scatter(x='Lat',y='Long', c=labels, s=50, cmap='spring', figsize=(20,20))
plt.scatter(centers[:, 0], centers[:, 1], c='black', s= 200, alpha=.5)

plt.show()

In [ ]:
#kmeans using the set cluster count
kmeans = KMeans(n_clusters=14, init='k-means++')
cluster_df = coord_df
kmeans.fit(cluster_df)
cluster_df['cluster'] = kmeans.fit_predict(cluster_df)
inertia = kmeans.inertia_
labels = kmeans.predict(cluster_df[cluster_df.columns[:]])
centers = kmeans.cluster_centers_

cluster_df.plot.scatter(x='Lat',y='Long', c=labels, s=50, cmap='spring', figsize=(40,40))
plt.scatter(centers[:, 0], centers[:, 1], c='black', s= 200, alpha=.5)

plt.show()

In [ ]:
centers_df = pd.DataFrame(data=centers, columns=["Lat", "Long", "delete"])
centers_df = centers_df.iloc[:,:2]
ln = pd.read_csv('./Irma Storm centered/ATL_17_11_Irma_WWLLN_Locations.txt',header=None,names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"],low_memory=False,sep=' ')
minLong_, minLat_, maxLong_, maxLat_ = ln['Long'].min(), ln['Lat'].min(), ln['Long'].max(), ln['Lat'].max()
buffer = 10

In [ ]:
plt.figure(figsize=(20,20))
m = Basemap(llcrnrlon=minLong_-buffer, llcrnrlat=minLat_-buffer,urcrnrlon=maxLong_+buffer,urcrnrlat=maxLat_+buffer,lon_0=0,lat_0=0)
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
m.drawcoastlines(linewidth=0.1, color="white")

# Plot the lightning data
m.plot(centers_df['Long'], centers_df['Lat'], linestyle='none', marker="o", markersize=75, alpha=0.7, c="orange", markeredgecolor="black", markeredgewidth=1)
plt.title("0 minutes")
plt.savefig("./data/test",bbox_inches='tight')